In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'
matplotlib.rcParams['savefig.dpi'] = 300
from matplotlib import cm

import seaborn as sns

from tqdm.auto import tqdm

import os
import ctypes
import gc

In [2]:
#  https://stackoverflow.com/a/37664693/13213091
def wrapped_ndptr(*args, **kwargs):
    base = np.ctypeslib.ndpointer(*args, **kwargs)
    def from_param(cls, obj):
        if obj is None:
            return obj
        return base.from_param(obj)
    return type(base.__name__, (base,), {'from_param': classmethod(from_param)})
DoubleArrayType_1D = wrapped_ndptr(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
DoubleArrayType_2D = wrapped_ndptr(dtype=np.float64, ndim=2, flags='C_CONTIGUOUS')

In [3]:
dirname = '../src/model_ctypes/_bondarenko'
filename_so = os.path.join(dirname, 'model.so')

filename_so_abs = os.path.abspath(filename_so)

model = ctypes.CDLL(filename_so_abs)

model.run.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'), # double *S
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'), # double *C
    ctypes.c_int, # int n_beats
    ctypes.c_double, # double t_sampling
    ctypes.c_double, # double tol
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=2, flags='C_CONTIGUOUS'), # double *output
    
    DoubleArrayType_2D, #np.ctypeslib.ndpointer(dtype=np.float64, ndim=2, flags='C_CONTIGUOUS'), # double *output_A
    DoubleArrayType_1D, #np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS') # double *output_t
    DoubleArrayType_1D, # double *stim_protocol
]

model.run.restype = ctypes.c_int


def run(S, C, n_beats, t_sampling, tol, output, *,
        output_A=None, t=None, stim_protocol=None):
    
    return model.run(S, C, n_beats, t_sampling, tol, output, output_A, t, stim_protocol)


legend_constants = pd.read_csv(os.path.join(dirname, "legend_constants.csv"), index_col='name')['value']
legend_states = pd.read_csv(os.path.join(dirname, "legend_states.csv"), index_col='name')['value']
# legend_algebraic = pd.read_csv(os.path.join(dirname, "legend_algebraic.csv"), index_col='name')
# legend_algebraic['value'] = 0.0
# legend_algebraic = legend_algebraic['value']

# model.initialize_states_default(legend_states.values, legend_constants.values)
# legend_states.to_csv(os.path.join(dirname, "legend_states.csv"))

In [4]:
S = legend_states.copy()
R = S * 0
C = legend_constants.copy()

In [5]:
t_sampling = 1

CL = 250
C['CL'] = CL
stim_period = C['CL']
n_samples_per_stim = int(stim_period / t_sampling)

n_beats = 1
tol = 1e-5

In [10]:
%%timeit

output = np.zeros((n_samples_per_stim * n_beats + 1, len(S)))

status = run(S.values.copy(), C.values.copy(),
             n_beats, t_sampling, tol, output)

#print(status)

output = pd.DataFrame(output, columns=legend_states.index)

154 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
output = np.zeros((n_samples_per_stim * n_beats + 1, len(S)))

In [13]:
t_sampling = 1

CL = 250
C['CL'] = CL
stim_period = C['CL']
n_samples_per_stim = int(stim_period / t_sampling)

n_beats = 1
tol = 1e-5

In [14]:
%%timeit

status = run(S.values.copy(), C.values.copy(),
             n_beats, t_sampling, tol, output)

154 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
plt.plot(output.V)

# RHS benchmarking

In [7]:
ydot = np.zeros_like(S)
y = S.values.copy()
params = C.values.copy()
t = 0.42

In [8]:
%%timeit
model.fun(t, y, ydot, params)

ArgumentError: argument 1: <class 'TypeError'>: Don't know how to convert parameter 1

In [ ]:
def fun(t, y, params):
    ydot = np.zeros_like(y)
    model.fun(t, y, ydot, params)
    # ydot[[7, 16, 19, 118]] = 0  # these are stiff guys
    return ydot

In [ ]:
%%timeit
fun(t, y, params)

# Generate synthetic  baselines

In [ ]:
CL_list = [1000, 500, 250]

for CL in tqdm(CL_list):

    C['CL'] = CL
    n_beats = 10
    stim_period = CL
    t_sampling = 1.

    t_space = np.linspace(0, stim_period * n_beats, int(stim_period / t_sampling) * n_beats + 1, endpoint=True)
    t_span = 0, t_space[-1]
    
    sol = solve_ivp(fun, y0=S,
                t_span=t_span, t_eval=t_space,
                args=(C.values.copy(),),
                method='LSODA',# rtol=1e-9,
                max_step=1. * t_sampling,
                )
    
    t, y = sol.t, sol.y
    v = y[1]
    
    v = v[-CL - 1: -CL - 1 + 250]
    
    folder_name = "../data/bondarenko/syn/"
    
    filename_phenotype = os.path.join(folder_name, f'phenotypes/phenotype_{CL}.csv')
    df = pd.DataFrame(v, columns=['V'])
    df.to_csv(filename_phenotype, index=False)
    
    filename_state = os.path.join(folder_name, f'states/state_{CL}.txt')
    np.savetxt(filename_state, y[:, -1])

In [ ]:
CL = 250

C['CL'] = CL
n_beats = 1
stim_period = CL
t_sampling = 1.

t_space = np.linspace(0, stim_period * n_beats, int(stim_period / t_sampling) * n_beats + 1, endpoint=True)
t_span = 0, t_space[-1]

In [ ]:
#%%timeit
sol = solve_ivp(fun, y0=S,
                t_span=t_span,# t_eval=t_space,
                args=(C.values.copy(),),
                method = 'LSODA', # atol=atol, # rtol=1e-3,
                #min_step=1e-3,
                #max_step=1. * t_sampling,
                )

In [ ]:
# %%time
# sol = solve_ivp(fun, y0=S,
#                 t_span=t_span, t_eval=t_space,
#                 args=(C.values.copy(),),
#                 method='LSODA',
#                 max_step=1. * t_sampling,
#                 )

In [ ]:
t, y = sol.t, sol.y
output = pd.DataFrame(y.T, columns=legend_states.index)

In [ ]:
output

In [ ]:
plt.figure()
plt.plot(t[1:], output['V'][1:])

In [ ]:
plt.figure(dpi = 100)
#plt.semilogy(t[:], sol.t[:], '.-')
plt.hist(sol.t, bins = 250)
plt.show()

In [ ]:
for i, c in enumerate(output):
    plt.plot(t, output[c])
    plt.scatter(t[1:], output[c][:-1], s=(np.log(np.diff(t)))**2, alpha=0.1, color='C3')
    plt.title(f'{i}\n{c}')
    # plt.xlim(10, 16)
    plt.show()

# tol

In [ ]:
atol = (output.abs().min() / 10).clip(lower=1e-6)
atol = np.power(10, np.log10(atol).round())

s = ""
for i, name in enumerate(atol.index):
    x = f'/*{name}*/ {atol[name]}, '
    s += x
    s += (20 - len(x)) * ' '
    if i % 5 == 4:
        s += '\n'

In [ ]:
print(s)

In [ ]:
atol_list = []
tol = 1e-6

for c in output:
    
    x = output[c]
    ptp = np.ptp(x)
    
    if ptp > 0.5:
        atol_list.append(tol)
        
    elif 0 < ptp <= 0.5:
        
        m = np.min(np.abs(x))
        
        if m > 0:
            atol_list.append(m / 10)
            
        else:
            atol_list.append(np.max(np.abs(x)) / 1e3)
            
    else:
        atol_list.append(42)
        
    continue
    if atol_list[-1] < 1e-9:
        atol_list[-1] = 1e-9